<a href="https://colab.research.google.com/github/kamgachristelle/SN-GROUPE-5/blob/main/pr%C3%A9diction_du_mis_par_un_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code de Génération de Données

In [ ]:
import pandas as pd
import numpy as np

# --------------------------------------------------------------------------------------
# TITRE : Définition des paramètres de taille et de taux d'échec pour le jeu de données.
# --------------------------------------------------------------------------------------
NB_LIGNES = 2057  # Nombre total de lignes (2000 + 57 erreurs supplémentaires)
NB_FAILURES_TARGET = 357 # Nous garantissons ce nombre exact d'échecs (300 initial + 57)
NB_SUCCESSES_TARGET = NB_LIGNES - NB_FAILURES_TARGET # 1700 réussites


# --------------------------------------------------------------------------------------
# TITRE : Génération des lignes simulant les exécutions réussies (83% du total).
# --------------------------------------------------------------------------------------
# Données de SUCCÈS (Status = success)
data_success = pd.DataFrame({
    'status': ['success'] * NB_SUCCESSES_TARGET,
    'code_lines_changed': np.random.randint(10, 800, size=NB_SUCCESSES_TARGET)
})
data_success['pred_risk_fail'] = np.random.uniform(0.01, 0.15, size=NB_SUCCESSES_TARGET)
data_success['error_tag'] = 'None'


# --------------------------------------------------------------------------------------
# TITRE : Génération des lignes simulant les échecs (17% du total) pour enrichir l'analyse d'erreur.
# --------------------------------------------------------------------------------------
# Données d'ÉCHECS (Status = failure)
data_failure = pd.DataFrame({
    'status': ['failure'] * NB_FAILURES_TARGET,
    'code_lines_changed': np.random.randint(50, 900, size=NB_FAILURES_TARGET) # Les échecs peuvent avoir des changements plus grands
})
data_failure['pred_risk_fail'] = np.random.uniform(0.65, 0.95, size=NB_FAILURES_TARGET)
# Répartition des types d'erreurs
data_failure['error_tag'] = np.random.choice(['TestFail', 'DependencyError', 'Timeout', 'AuthIssue'], size=NB_FAILURES_TARGET)


# --------------------------------------------------------------------------------------
# TITRE : Consolidation des données Succès/Échec en un seul jeu de données mélangé.
# --------------------------------------------------------------------------------------
# COMBINER et mélanger les jeux de données
data = pd.concat([data_success, data_failure]).reset_index(drop=True)
np.random.shuffle(data.values)


# --------------------------------------------------------------------------------------
# TITRE : Attribution d'un ID unique, d'un horodatage et des variables catégorielles (branches, dépôts).
# --------------------------------------------------------------------------------------
# Ajout des colonnes temporelles et IDs (pour 2057 lignes)
data['workflow_id'] = [f'W-{i:04d}' for i in range(1, NB_LIGNES + 1)]
start_time_base = pd.to_datetime('2025-10-20')
# Étalement temporel
data['start_time'] = start_time_base + pd.to_timedelta(np.arange(NB_LIGNES) * 500 + np.random.randint(0, 100, NB_LIGNES), unit='s')

# Ajout des autres colonnes
data['branch_name'] = np.random.choice(['main', 'develop', 'feature-auth', 'feature-payment', 'hotfix'], size=NB_LIGNES, p=[0.3, 0.3, 0.2, 0.1, 0.1])
data['repo_name'] = np.random.choice(['repo-api', 'repo-front', 'repo-mobile'], size=NB_LIGNES, p=[0.5, 0.3, 0.2])
data['env_type'] = np.random.choice(['prod', 'staging', 'dev'], size=NB_LIGNES, p=[0.2, 0.3, 0.5])


# --------------------------------------------------------------------------------------
# TITRE : Calcul de la durée réelle et simulation de la prédiction du modèle ML.
# --------------------------------------------------------------------------------------
# Durée réelle (proportionnelle aux lignes de code + bruit)
data['duration_s'] = (data['code_lines_changed'] * 0.8) + np.random.randint(60, 180, size=NB_LIGNES)
data['duration_s'] = data['duration_s'].astype(int)

# Prédiction de Durée
data['pred_duration_s'] = data['duration_s'] * np.random.uniform(0.95, 1.05, size=NB_LIGNES)
data['pred_duration_s'] = data['pred_duration_s'].astype(int)


# --------------------------------------------------------------------------------------
# TITRE : Sauvegarde finale des données préparées au format CSV pour l'import dans Grafana.
# --------------------------------------------------------------------------------------
nom_fichier = 'ci_cd_prediction_mock_2057_errors.csv'
data = data[['workflow_id', 'start_time', 'duration_s', 'status', 'pred_duration_s', 'pred_risk_fail', 'branch_name', 'repo_name', 'code_lines_changed', 'env_type', 'error_tag']]
data.to_csv(nom_fichier, index=False, date_format='%Y-%m-%d %H:%M:%S')

print(f"✅ Fichier '{nom_fichier}' de {NB_LIGNES} lignes généré avec 357 échecs garantis.")

# TITRE : Bilan de Santé Initial du Jeu de Données
Ce bloc charge le fichier CSV et vérifie que les types de données sont corrects pour l'analyse et le Machine Learning.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Charger le jeu de données (2057 lignes)
df = pd.read_csv('ci_cd_prediction_mock_2057_errors.csv')

# Convertir la colonne start_time au format datetime
df['start_time'] = pd.to_datetime(df['start_time'])

print("--- Bilan de Santé des Données (2057 lignes) ---")
print(f"Nombre total de lignes : {len(df)}")
print(f"Aperçu des types de données :")
print(df.dtypes)

# Proportion Succès/Échec (Validation du Déséquilibre)
Ce bloc calcule et visualise le taux de déséquilibre de la variable cible (status )près l'ajout des 57 erreurs.

In [ ]:
# Diagramme : Analyse du Déséquilibre Succès/Échec
status_counts = df['status'].value_counts()
status_percentages = df['status'].value_counts(normalize=True) * 100

print("\n--- Fréquence des Statuts de Pipeline ---")
print(status_counts)
print("\n--- Pourcentage ---")
print(status_percentages)

# Visualisation des pourcentages (Camembert)
plt.figure(figsize=(7, 7))
plt.pie(
    status_counts,
    labels=status_counts.index,
    # Afficher le pourcentage avec une décimale
    autopct='%1.1f%%',
    startangle=90,
    colors=['#4CAF50', '#FF5722']
)
plt.title('Proportion Succès vs Échec (Validation du Déséquilibre)')
plt.show()
#

# importation de ydata-profilingest la première étape pour  analyse exploratoire

In [ ]:
import sys

# Installer la bibliothèque ydata-profiling
!{sys.executable} -m pip install ydata-profiling

# Génération du Rapport de Profilage Interactif (Contrôle Qualité IA)
Ce bloc importe la librairie YData-Profiling pour créer un rapport HTML complet du jeu de données ( df). Ce rapport est utilisé pour l' analyse exploratoire (EDA) , le contrôle de la qualité, la détection des valeurs manquantes et l'identification des corrélations, garantissant la fiabilité des données avant l'entraînement du modèle de Machine Learning.

In [ ]:
from ydata_profiling import ProfileReport

# Générer le rapport de profilage
profile = ProfileReport(df, title="Rapport de Profilage des Données CI/CD", html={
    'style': {'full_width': True},
    'minify': True,
    'inline': True,
    'use_local_assets': True
}, explorative=True, lazy=False)

# Sauvegarder le rapport dans un fichier HTML
output_file = "ci_cd_data_profiling_report_fr.html"
profile.to_file(output_file)

# Afficher le rapport directement dans le notebook
profile.to_notebook_iframe()

print(f"✅ Rapport de profilage généré et sauvegardé sous : '{output_file}'")

# Entraînement et Évaluation du Modèle Forêt Aléatoire 🌳

 Initialisation des Modules de Machine Learning et de Mesure
Ce bloc importe les outils nécessaires à la modélisation et à l'évaluation, y compris les modèles de Régression Linéaire et Forêt Aléatoire, ainsi que les métriques d'erreur (MAE, R2).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd

 Préparation des Données et Encodage des Variables Catégorielles
Ce bloc prépare le jeu de données (df ) en :

Excluant les colonnes non prédictives ou cibles ( workflow_id, pred_duration_s, start_time).

Encodant les variables textuelles ( branch_name, repo_name, etc.) en colonnes numériques binaires (One-Hot Encoding), car les modèles ML ne peuvent traiter que des nombres.

In [ ]:
# Définition des variables cibles (y) et des variables explicatives (X)
colonnes_a_exclure = ['workflow_id', 'pred_duration_s', 'pred_risk_fail', 'start_time']
X = df.drop(columns=['duration_s'] + colonnes_a_exclure)
y = df['duration_s']

# Encodage des Variables Catégorielles
colonnes_categoriques = ['status', 'branch_name', 'repo_name', 'env_type', 'error_tag']
X_encoded = pd.get_dummies(X, columns=colonnes_categoriques, drop_first=True)

Séparation du Jeu de Données (Train/Test Split) et Entraînement du Modèle Forêt Aléatoire
Ce bloc divise les données encodées en deux parties (80% pour l'entraînement et 20% pour le test) et entraîne le modèle Random Forest Regressor , qui utilise une collection d'arbres de décision pour prédire la durée.

In [ ]:
# Séparation en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)
# Entraînement du modèle
model_rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_rf.fit(X_train, y_train)

Évaluation de la Performance du Modèle de Régression
Ce bloc utilise les données de test (que le modèle n'a jamais vues) pour vérifier sa précision en calculant deux métriques clés :

MAE (Erreur Absolue Moyenne) : L'erreur moyenne de la prédiction, exprimée en secondes.

R² (Coefficient R-carré) : La qualité de l'ajustement (plus la valeur est proche de 1.0000, mieux c'est).

In [ ]:
# Évaluation (pour confirmer la performance)
y_pred_rf = model_rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("\n--- Nouvelle Performance Forêt Aléatoire ---")
print(f"Erreur Absolue Moyenne (MAE): {mae_rf:.2f}")
print(f"Coefficient R-carré (R2): {r2_rf:.4f}")

# Modèle de Régression Linéaire

Initialisation et Entraînement du Modèle de Régression Linéaire
Ce bloc importe la classe LinearRegressionet lance le processus d'apprentissage. Le modèle est entraîné sur l'ensemble X_train(caractéristiques encodées) pour prédire la cible variable y_train(durée réelle en secondes)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# 1. Initialisation et entraînement du modèle Linéaire
model_linear = LinearRegression()
model_linear.fit(X_train, y_train)

Prédiction et Évaluation de la Performance (Baseline)
Ce bloc utilise le modèle entraîné pour prédire la durée sur les données de test ( X_test) que le modèle n'a jamais vues, puis calculer les métriques pour évaluer sa précision.

In [ ]:
# 2. Prédiction sur l'ensemble de test
y_pred_linear = model_linear.predict(X_test)

# 3. Évaluation de la performance
mae = mean_absolute_error(y_test, y_pred_linear)
r2 = r2_score(y_test, y_pred_linear)

print("\n--- Performance du Modèle de Régression Linéaire (Baseline) ---")
print(f"Erreur Absolue Moyenne (MAE): {mae:.2f}")
print(f"Coefficient R-carré (R2): {r2:.4f}")

# TITRE : Extraction et Interprétation des Coefficients d'Influence
Ce bloc de code importe la librairie Pandas pour structurer les résultats. Il extrait les coefficients ($\beta$) calculés par la Régression Linéaire ( model_linear.coef_) et les associés aux noms des colonnes encodées ( X_train.columns). Il trie les résultats pour identifier les 10 facteurs qui ont le plus grand impact direct (positif ou négatif) sur la durée du pipeline.

In [ ]:
import pandas as pd

# Créer un DataFrame pour les coefficients
coefficients = pd.Series(
    model_linear.coef_,
    index=X_train.columns
).sort_values(ascending=False)

print("\n--- Top 10 des Coefficients d'Influence (Régression Linéaire) ---")
print(coefficients.head(10))

# TITRE : Sauvegarde et Sérialisation du Modèle Final (Persistance MLOps)
Ce bloc de code utilise la librairie joblibpour enregistrer l'objet Python entraîné ( model_linear) sur le disque local dans un format binaire ( .joblib). Cette étape est fondamentale car elle permet de conserver l'état mathématique du modèle (les coefficients calculés) après la fermeture du Notebook, et de le rendre portable pour un futur déploiement (par exemple, sur Amazon SageMaker).

In [ ]:
import joblib
import os
import numpy as np

# 1. Définir le nom du fichier de sauvegarde
model_filename = 'linear_regression_final_model.joblib'

# 2. Sauvegarder l'objet Python 'model_linear' sur le disque
# Le modèle entraîné est maintenant un fichier binaire
joblib.dump(model_linear, model_filename)

print(f"✅ Modèle sauvegardé avec succès sous : '{model_filename}'")

# Prédiction de la Durée d'un Nouveau Pipeline (Inférence ML)
Ce bloc de code exécute l'inférence (la prédiction) en temps réel en utilisant le modèle de Régression Linéaire sauvegardé. Le processus garantit que la nouvelle donnée de pipeline est encodée et alignée (One-Hot Encoding, même ordre de colonnes) pour correspondre parfaitement au format attendu par le modèle adopté, simulant ainsi un appel API vers un point final SageMaker.

In [ ]:
import joblib
import pandas as pd
import numpy as np

# 1. Charger le modèle sauvegardé
model_loaded = joblib.load('linear_regression_final_model.joblib')

# Récupérer les colonnes de X_train pour s'assurer de l'ordre et des colonnes présentes
feature_columns = X_train.columns

# Définir les variables catégorielles (doit correspondre à la liste utilisée pour l'entraînement)
colonnes_categoriques = ['status', 'branch_name', 'repo_name', 'env_type', 'error_tag']

# --- Préparation de la nouvelle donnée pour la prédiction ---

# 2. Créer un DataFrame pour le nouveau point de donnée avec les colonnes originales
new_observation_raw = pd.DataFrame({
    'code_lines_changed': [500],
    'status': ['success'], # Valeur réelle pour 'status'
    'branch_name': ['develop'], # Valeur réelle pour 'branch_name'
    'repo_name': ['repo-api'], # Valeur réelle pour 'repo-api'
    'env_type': ['dev'], # Valeur réelle pour 'env_type'
    'error_tag': ['None'] # Valeur réelle pour 'error_tag'
})

# 3. Appliquer le One-Hot Encoding au nouveau point de donnée, avec drop_first=True
new_observation_encoded = pd.get_dummies(new_observation_raw, columns=colonnes_categoriques, drop_first=True)

# 4. Aligner les colonnes du nouveau point de donnée encodé avec les colonnes utilisées pour l'entraînement (X_train)
# Les colonnes manquantes seront remplies avec 0, et les colonnes supplémentaires (si elles apparaissent en raison de get_dummies)
# qui n'étaient pas dans X_train seront supprimées.
new_data_point = new_observation_encoded.reindex(columns=feature_columns, fill_value=0)

# Assurez-vous que toutes les colonnes numériques sont du bon type pour correspondre à X_train
for col in feature_columns:
    if col in new_data_point.columns:
        new_data_point[col] = new_data_point[col].astype(X_train[col].dtype)

# 5. Exécuter la prédiction sur la nouvelle ligne de données
predicted_duration = model_loaded.predict(new_data_point)

# 6. Afficher le résultat
print("\n--- Résultat de la Prédiction ---")
print(f"Durée prédite du pipeline avec 500 lignes de changement : {predicted_duration[0]:.2f} secondes")
print(f"Soit environ {predicted_duration[0] / 60:.2f} minutes.")